In [2]:
import pandas as pd

df = pd.read_csv("Churn_Modelling.csv")

In [3]:
import pickle

col_trans = None
with open("col_trans.pkl", "rb") as file:
    col_trans = pickle.load(file)

In [4]:
from sklearn.model_selection import train_test_split

X = df.drop("Exited", axis=1)
y = df["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [5]:
X_train.shape

(8000, 13)

In [6]:
X_train = col_trans.fit_transform(X_train)
X_test = col_trans.transform(X_test)

In [7]:
X_train.shape

(8000, 12)

In [8]:
X_train[0]

array([ 0.        ,  1.        ,  0.        ,  0.        , -0.54314547,
        2.0207228 , -1.73524323,  0.88733828, -0.91218445, -1.54081269,
       -1.02840321, -1.55151018])

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [10]:
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1], )),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [13]:
# callbacks
log_dir = "logs/fit"

tensorboard_callback = TensorBoard(log_dir)
early_stopping_callback = EarlyStopping(monitor="accuracy", patience=7, restore_best_weights=True)

In [14]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=25,
    callbacks=[
        tensorboard_callback,
        early_stopping_callback
    ]
)

Epoch 1/25


250/250 [==============================] - 3s 5ms/step - loss: 0.4509 - accuracy: 0.8037 - val_loss: 0.3916 - val_accuracy: 0.8450
Epoch 2/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3680 - accuracy: 0.8501 - val_loss: 0.3551 - val_accuracy: 0.8565
Epoch 3/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3503 - accuracy: 0.8551 - val_loss: 0.3482 - val_accuracy: 0.8545
Epoch 4/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3439 - accuracy: 0.8579 - val_loss: 0.3450 - val_accuracy: 0.8560
Epoch 5/25
250/250 [==============================] - 1s 2ms/step - loss: 0.3410 - accuracy: 0.8574 - val_loss: 0.3407 - val_accuracy: 0.8565
Epoch 6/25
250/250 [==============================] - 1s 2ms/step - loss: 0.3373 - accuracy: 0.8601 - val_loss: 0.3373 - val_accuracy: 0.8560
Epoch 7/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3341 - accuracy: 0.8619 - val_loss: 0.3385 - val_accuracy: 0.8595
Epoc

In [15]:
model.save("model.keras")